In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium import IFrame
from folium import Map, FeatureGroup, Marker, LayerControl, Popup

In [ ]:
# read in CSV from webscrape
cummins_df = pd.read_csv('/home/nhac/Documents/NSS/Python/capstone/cummins_scrape_1.csv')
cummins_df.head()

In [ ]:
# Build loop to grab columns and data from cummins_df and output to dataframe

fruit_list = pd.DataFrame()     # create empty dataframe to append results
for i in range(3,10):     # iterate through numbers to assign for zone
    for j in ['Apple','Peach','Plum','Pear','Nectarine','Cherry','Apricot']:     # for every fruit type names
        col_name = j + str(i)     # concat zone number to fruit type name to get matching column name in cummins_df (Apple3, Apple4, etc.)
        tmp_f_list = cummins_df[[col_name]].copy()     # tmp list with column name matching column name (variable)
        tmp_f_list = tmp_f_list[tmp_f_list[col_name].notna()]     # drop the NaNs from the list for later list concat
        tmp_f_list = tmp_f_list.rename(columns={col_name:'fruit_name'})     # rename the variable column name, with static name
        tmp_f_list['zone'] = i     # add a column to the tmp DF and populate the column with variable RANGE number
        tmp_f_list[['type']] = j     # add a column to the tmp DF and populate with variable fruit type name
        fruit_list = pd.concat([fruit_list, tmp_f_list])   # append results from tmp_f_list to previously empty df, fruit_list

In [ ]:
# reset index
fruit_list = fruit_list.reset_index(drop=True)
fruit_list.head()

In [ ]:
# read in hardiness zones geojson
geo_h_zones = gpd.read_file('../data/us-hardiness-zones_1291.geojson')

In [ ]:
# normalize column names for merge
geo_h_zones = geo_h_zones.rename(columns={'name':'zone'})

In [ ]:
# CRS to EPSG
geo_h_zones = geo_h_zones.to_crs(epsg=4326)

In [ ]:
# check for GeoDataFrame
type(geo_h_zones)

In [ ]:
# convert polys to centroids from geojson
centroid_df = geo_h_zones.geometry.centroid.to_frame(name='centroid')

In [ ]:
# join geojson with newly created centroids
geo_df = pd.concat([geo_h_zones, centroid_df] , axis=1)

In [ ]:
# normalize 'zone' column type to int for merge
geo_df.zone = geo_df.zone.astype(int)

In [ ]:
# merge fruit data with geo data on 'zone'
all_frame = pd.merge(fruit_list, geo_df, how='inner', on='zone')

In [ ]:
all_frame

In [ ]:
#all_frame.to_csv('../data/fruit_var_hardy.csv')

In [ ]:
# Extract lat and lon from centroid, insert into new columns
all_frame['lon'] = all_frame.centroid.apply(lambda p: p.x)
all_frame['lat'] = all_frame.centroid.apply(lambda p: p.y)

In [ ]:
all_frame.centroid = all_frame.centroid.astype(str)

In [ ]:
z3 = all_frame[all_frame['zone'] == 3]

In [ ]:
maps_fruit_list = all_frame.drop_duplicates(subset=['fruit_name','zone','type'])

In [ ]:
choro_count = maps_fruit_list.groupby('zone').count().reset_index()

In [ ]:
choro_count.zone = choro_count.zone.astype(str)

In [ ]:
choro_count = pd.merge(geo_h_zones, choro_count, how='inner', on='zone')

In [ ]:
choro_count.geometry = choro_count.geometry_x

In [ ]:
geometry = geo_h_zones['geometry']
simplified_geometry = geometry.simplify(tolerance=0.125)
geo_h_zones['geometry'] = simplified_geometry

In [ ]:
# read in state lines geojson
state_lines = gpd.read_file('/home/nhac/Documents/NSS/Python/capstone/data/census_state_boundaries/cb_2021_us_state_500k.shp')
geojson2 = state_lines.to_crs(epsg=4326).to_json()

In [ ]:
state_lines['State'] = state_lines.NAME
state_lines.head()

In [ ]:
# read in Food Access Data xlsx
food_acc_raw = pd.read_excel('/home/nhac/Documents/NSS/Python/capstone/data/FoodAccessResearchAtlasData2019.xlsx', sheet_name=2)
# read population estimates
pop_est_raw = pd.read_csv('../data/pop-est2019-alldata.csv')

In [ ]:
pop_2019 = pop_est_raw[['STATE','NAME','POPESTIMATE2019']]

In [ ]:
# function to view column names in a dataframe
def getColumns(df):
    for col in df.columns:
        print(col)

In [ ]:
pop_2019.head()

In [ ]:
# merge food access data with state lines geodata
food_geo = pd.merge(state_lines, food_acc_raw, how='inner', on='State')

In [ ]:
low_acc_pop = food_geo.groupby('NAME')['LALOWI1_10'].sum().astype(int).to_frame()


In [ ]:
states_pop = pd.merge(pop_2019, low_acc_pop, on='NAME', how='inner')

In [ ]:
states_pop['pct'] = states_pop['LALOWI1_10'] / states_pop['POPESTIMATE2019'] * 100

In [ ]:
states_pop

In [ ]:
state_pct_df = pd.merge(state_lines, states_pop, on='NAME', how='inner' )

In [ ]:
gdf = gpd.GeoDataFrame(state_pct_df, geometry='geometry')

In [ ]:
#gdf.to_csv('../data/pct_LALOWI1_10.csv')

In [ ]:
import mapclassify   # use this for quantile separation for visual clarity
import geoplot as gplt
import geoplot.crs as gcrs

In [ ]:
min_lon, max_lon = -125, -65
min_lat, max_lat = 24, 50
ax = gplt.choropleth(gdf,
                   hue="pct",
                   #scheme=mapclassify.Quantiles(fruit_intake_gdf["pct_rec_f_in"], k=4),
                   cmap="gist_earth", #edgecolor="grey",
                   extent=(-172,10, 10,75),
                   projection=gcrs.WebMercator(),
                   #legend_labels=["Least Met Recommended", "Somewhat Met", "Better","Best"],
                   #legend_kwargs={"loc":"best",
                   #               "fontsize": "small",
                   #              "title":"% of Sample Size",
                   #               "title_fontsize":"medium"},
                   figsize=(8,6), legend=True);

gplt.polyplot(state_lines, 
              linewidth=0.8,
              ax=ax
)
ax.set_extent((min_lon, max_lon, min_lat, max_lat))
ax.set_facecolor('#2499BE')
#ax.set_axis_bgcolor("#2499BE")

plt.title("Percent of State Population with Low Access and Income", 
          fontdict={"fontsize": 14}, 
          pad=12,
          color='#4c4c4c');

In [ ]:
pops = states_pop.groupby('NAME')['LAPOP1_10'].sum().astype(int).to_frame().reset_index()

### Folium

In [ ]:
# Create a Folium map
m = folium.Map(location=(31.51073, -96.4247),
               #width=1024,
               #height=768,
               zoom_start=4, 
               tiles='OpenStreetMap', 
               #width="100%", 
               #height="100%",
               no_wrap=True,
               min_zoom=4,
               max_zoom=6
              )
# Add the polygons to the map (maybe redundant)
geojson = geo_h_zones.to_crs(epsg=4326).to_json()

# add chloropleth layer for hardiness zones
folium.Choropleth(
    geo_data=geojson,
    name='Hardiness Zones',
    data=choro_count,
    columns=['zone','fruit_name'],
    fill_color='PuBuGn',
    fill_opacity=0.6,
    weight=3,
    key_on='feature.properties.zone',
    legend_name='Total Number of Fruit Types',
    legend_loc='upper left'
).add_to(m)

# add choropleth layer for food insecurity states
folium.Choropleth(
    geo_data=geojson2,
    name='Low Access and Income',
    data=gdf,
    columns=["NAME", "pct"],
    key_on='feature.properties.NAME',
    fill_color='YlOrRd',
    fill_opacity=0.6,
    line_opacity=0.2,
    legend_name="Low Income and Access (Urb 1mi, Rur 10mi from grocery)",
).add_to(m)

# Iterate through the "zone" column of the data frame
for z in maps_fruit_list['zone']:
    # Select rows that have the current category
    items = maps_fruit_list[maps_fruit_list['zone'] == z]
    
    # Place the items in an HTML table
    html_table = "<table style='width:200px'>"
    for i, row in items.iterrows():
        html_table += "<tr><td>{}</td><td>{}</td><td>{}</td></tr>".format(row['fruit_name'], row['type'], row['zone'])
    html_table += "</table>"

    # Add a marker to the map for the zone centroid    
    marker = folium.Marker(location=[row['lat'], row['lon']],
                           popup=folium.Popup(html_table, 
                                              max_height=300,
                                              scroll_wheel=True),
                          )
    marker.add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
m

In [ ]:
#m.save('../data/frt_lst_hardy.html')

In [ ]:
j = folium.Map(location=(31.51073, -96.4247), zoom_start=5, tiles='CartoDB positron')

for _, r in maps_fruit_list.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.2)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    
    folium.Popup(r['zone']).add_to(geo_j)
    geo_j.add_to(m)


for _, r in maps_fruit_list.iterrows():
    lat = r['centroid'].y
    lon = r['centroid'].x
    folium.Marker(location=[lat, lon]),
j

In [ ]:
maps_fruit_list.head()

### Geoplot

In [ ]:
# map it using geoplot
import mapclassify   # use this for quantile separation for visual clarity
import geoplot as gplt
import geoplot.crs as gcrs
# set min and max lon and lat for USA map extents (zoom in)
min_lon, max_lon = -125, -65
min_lat, max_lat = 24, 50
cmap = plt.cm.YlGn
# plot map
ax = ax = gplt.webmap(food_geo,
                 figsize=(16, 8),
                 projection=gcrs.WebMercator())
 
gplt.choropleth(choro_count,
                   hue="fruit_name",
                   scheme=mapclassify.Quantiles(choro_count['fruit_name'], k=10),
                   cmap=cmap,
                   alpha=0.6,
                   #edgecolor="white",
                   extent=(-172,10, 10,75),
                   projection=gcrs.AlbersEqualArea(),
                   #legend_labels=['Zone 2','Zone 3','Zone 4','Zone 5','Zone 6','Zone 7','Zone 8','Zone 9'],
                   legend_kwargs={"loc":'lower right',
                                  "fontsize": "small",
                                  "title":"Number of Fruit Types",
                                  "title_fontsize":"medium"},
                   #figsize=(16,8), 
                   legend=True,
                   ax = ax)

gplt.polyplot(state_lines, 
              linewidth=0.5,
              ax=ax
);

ax.set_extent((min_lon, max_lon, min_lat, max_lat))
ax.set_facecolor('lightgrey')
plt.title("Regional Fruit Varieties Possible", 
          fontdict={"fontsize": 14},
          color='#4c4c4c',
          pad=5);

plt.savefig('../data/choropleth.png', dpi=300)


### Plotly

In [ ]:
gdf.head()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import json

In [ ]:
choro_count.head()

In [ ]:
data = gdf
with open('../data/gz_2010_us_040_00_500k.json') as f:
    geo_json = json.load(f)
    
fig = px.choropleth(data, locations="STUSPS", color="pct",
                    locationmode="USA-states", 
                    geojson=geo_json, 
                    scope='usa',
                    title="Low Income and Low Access Population",
                    color_continuous_scale=px.colors.sequential.RdPu,
                    hover_data=['NAME','POPESTIMATE2019'],
                    hover_name='NAME')

fig.update_layout(height=768, width=1024)

fig.update_layout(titlefont=dict(size=28, family='Droid Sans Mono', color='#000000'))

fig.update_layout(coloraxis_colorbar=dict(
    title="% of State Population",
    titlefont=dict(
        family="Droid Sans Mono",
        size=16,
        color="#000000"
    )
))
fig.add_scattergeo(
    locations=data['STUSPS'],
    locationmode='USA-states',
    text=data['STUSPS'],
    mode='text')

fig.update_layout(
    hoverlabel=dict(
        font_size=16,
        font_color="#ffffff",
        font_family='Droid Sans Mono'
    )
)
fig.show()

In [ ]:
import plotly.io as pio
pio.write_image(fig,'../data/LALOWI1_10.png')